# Modelo de un capacitor con condiciones

Resolver la ecuación de Laplace en dos dimensiones para el potencial electroestático $\phi = \phi(x,y)$:

$$ \frac{\partial\phi^2}{\partial x^2} + \frac{\partial\phi^2}{\partial y^2} = 0 $$

para una placa cuadrada de 10cm x 10cm, con dadas condiciones iniciales/de frontera.

In [ ]:
#Cómo generar la placa con las dimensiones correctas y las condiciones iniciales

import numpy as np
import matplotlib.pyplot as plt

prop = 1 #Proporción de los lados de la placa
plate = np.zeros((10*prop,10*prop+1), dtype=float)
plate[2*prop:8*prop,2*prop]=1.0
plate[2*prop:8*prop,8*prop]=-1.0

print(plate)
plt.imshow(plate)
plt.gray()
plt.show()

In [ ]:
#Utilizando el método de Jacobi

def jacobi(tolerance, prop, V1, V2): #Se define con una tolerancia y una proporción de la placa a utilizar
        
    plate = np.zeros((10*prop,10*prop+1), dtype=float)     #Genera una placa (matriz) 1:1, del tipo 10n x 10n
                                                             #donde "n" es un parámetro de proporción, para poder 
                                                             #ejecutarlo a diferentes escalas
    platePrime = plate.copy()     #Genera una copia de la grilla anterior.
    #Condición inicial
    plate[2*prop:8*prop,2*prop]=V1     #Genera la franja de potencial V = V1
    plate[2*prop:8*prop,8*prop]=V2     #Genera la franja de potencial V = V2
    #Iteraciones de Jacobi
    diff = 1.0     #Establece una diferencia inicial, que irá reduciéndose hasta alcanzar la tolerancia
    iterations = 0 
    while diff > tolerance:
        iterations += 1     #Registrar número de iteraciones
        for i in range(10*prop-1):     #Iterar por filas
            for j in range(1,10*prop):     #Iterar por columnas
                #Condición de frontera
                if 2*prop <= i < 8*prop and (j == 2*prop or j == 8*prop):     #Mantener franjas invariantes
                    platePrime[i,j] = plate[i,j]
                else:
                    platePrime[i,j] = (plate[i+1,j]+plate[i-1,j]+plate[i,j+1]+plate[i,j-1])*(0.25)
        diff = np.max(np.abs(plate - platePrime))     #Determina la máxima diferencia entre la grilla nueva y la
                                                      #iteración anterior. Continúa si el valor es mayor a la tolerancia
        temp = plate     #Matriz temporal, para intercambiar plate <--> platePrime
        plate = platePrime
        platePrime = temp
    
    return platePrime, iterations

In [ ]:
prop_values = [1,2,3,5,10]
graphicsJacobi = np.empty(len(prop_values), dtype=object)
for i in range(len(prop_values)):
    graphicsJacobi[i] = jacobi(1.0e-5, prop_values[i], 1.0, -1.0)
    plt.imshow(graphicsJacobi[i][0], cmap = "gray")
    plt.tight_layout()
    plt.title(f"Gráfico para proporción n = {prop_values[i]}")
    plt.show()
    print(f"Número de iteraciones: {graphicsJacobi[i][1]}")

In [ ]:
#Utilizando el método de Jacobi mejorado:

def jacobi_mejorado(tolerance, prop, V1, V2, omega): #Se define con una tolerancia y una proporción de la placa a utilizar
        
    plate = np.zeros((10*prop,10*prop+1), dtype=float)     #Genera una placa (matriz) 1:1, del tipo 10n x 10n
                                                             #donde "n" es un parámetro de proporción, para poder 
                                                             #ejecutarlo a diferentes escalas
    platePrime = plate.copy()     #Genera una copia de la grilla anterior.
    #Condición inicial
    plate[2*prop:8*prop,2*prop]=V1     #Genera la franja de potencial V = V1
    plate[2*prop:8*prop,8*prop]=V2     #Genera la franja de potencial V = V2
    #Iteraciones de Jacobi
    diff = 1.0     #Establece una diferencia inicial, que irá reduciéndose hasta alcanzar la tolerancia
    iterations = 0 
    while diff > tolerance:
        iterations += 1     #Registrar número de iteraciones
        for i in range(10*prop-1):     #Iterar por filas
            for j in range(1,10*prop):     #Iterar por columnas
                #Condición de frontera
                if 2*prop <= i < 8*prop and (j == 2*prop or j == 8*prop):     #Mantener franjas invariantes
                    platePrime[i,j] = plate[i,j]
                else:
                    platePrime[i,j] = (1.0+omega)*(plate[i+1,j]+plate[i-1,j]+plate[i,j+1]+plate[i,j-1])*0.25 - omega*plate[i,j]
        diff = np.max(np.abs(plate - platePrime))     #Determina la máxima diferencia entre la grilla nueva y la
                                                      #iteración anterior. Continúa si el valor es mayor a la tolerancia
        temp = plate     #Matriz temporal, para intercambiar plate <--> platePrime
        plate = platePrime
        platePrime = temp
    
    return platePrime, iterations

In [ ]:
plateM, itsM = jacobi_mejorado(1e-5, 1, 1.0, -1.0, 0.1)
plt.imshow(plateM)
plt.gray()
plt.show()
print(f"Número de iteraciones: {itsM}")

#Falla para proporciones mayores a 1

In [ ]:
# Utilizando el método de Gauss

def gaussS(tolerance, prop, V1, V2, omega): #Se define con una tolerancia y una proporción de la placa a utilizar
        
    plate = np.zeros((10*prop,10*prop+1), dtype=float)     #Genera una placa (matriz) 1:1, del tipo 10n x 10n
                                                             #donde "n" es un parámetro de proporción, para poder 
                                                             #ejecutarlo a diferentes escalas
    
    #Condición inicial
    plate[2*prop:8*prop,2*prop]=V1     #Genera la franja de potencial V = V1
    plate[2*prop:8*prop,8*prop]=V2     #Genera la franja de potencial V = V2
    #Iteraciones de Jacobi
    diff = 1.0     #Establece una diferencia inicial, que irá reduciéndose hasta alcanzar la tolerancia
    iterations = 0 
    while diff > tolerance:
        platePrime = plate.copy() #Genera una copia de la grilla anterior.
        iterations += 1     #Registrar número de iteraciones
        for i in range(10*prop-1):     #Iterar por filas
            for j in range(1,10*prop):     #Iterar por columnas
                #Condición de frontera
                if 2*prop <= i < 8*prop and (j == 2*prop or j == 8*prop):     #Mantener franjas invariantes
                    plate[i,j] = plate[i,j]
                else:
                    plate[i,j] = (1.0+omega)*(plate[i+1,j]+plate[i-1,j]+plate[i,j+1]+plate[i,j-1])*0.25 - omega*plate[i,j]
        diff = np.max(np.abs(plate - platePrime)) #Determina la máxima diferencia entre la grilla nueva y la
                                                  #iteración anterior. Continúa si el valor es mayor a la tolerancia
    
    return plate, iterations

In [ ]:
#Probando el resultado para distintos valores de omega, con una proporción constante de n=10 (placa 100x100)

prop_values = np.linspace(0.1, 0.9, 9)
for i in prop_values:
    plateG, itsG = gaussS(1.0e-5, 10, 1.0, -1.0, i)
    plt.imshow(plateG)
    plt.gray()
    plt.show()
    print(f"Número de iteraciones: {itsG}")

In [ ]:
# No logré que funcionara para omega > 0.9

plateG10, itsG = gaussS(1.0e-5, 10, 1.0, -1.0, 1.0)
plt.imshow(plateG10)
plt.gray()
plt.show()
print(f"Número de iteraciones: {itsG10}")